In [1]:
import pathlib
import yaml
ROOT = pathlib.Path().resolve().parent.parent

with open(ROOT / "config.yml", "r") as f:
    config = yaml.safe_load(f)

In [2]:
# 3
DATA_PATH = ROOT / config['api']['DATA_PATH'] / 'seminar4'

query = "корейская косметика"
type_query = "channel"

# Для поиска видео с учетом даты:
publishedAfter = '2020-01-01T00:00:00Z'
publishedBefore = '2022-01-01T00:00:00Z'

In [3]:
# 3.1
class KeyStorage:
    """
    Class to use in iteration
    """

    def __init__(self, key_arr: list[str]):
        self.key_gen = iter(key_arr)

    def get_next_key(self):
        """
        Returns API key to use
        :return:
        """
        return next(self.key_gen)

key_storage = KeyStorage(config['api']['keys'])

In [4]:
# 3.2 Просмотр API ключа
API_KEY = key_storage.get_next_key()

In [5]:
#4
import googleapiclient.discovery as api

In [6]:
#5
api_service_name = "youtube"
api_version = "v3"

youtube = api.build(api_service_name,
                        api_version,
                       developerKey = API_KEY)

In [8]:
# 7
import pandas as pd


data = pd.read_excel(DATA_PATH / 'Video_Not_sorted+Sorted+Date.xlsx')
data.head()

,Unnamed: 0,etag,id.kind,id.videoId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,snippet.thumbnails.medium.url,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.url,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime,kind
0,1,KC_LG8xTt9DjwtELdwXsHef8Y2g,youtube#video,Wi49rKRM85U,2022,UCphnZS52v_lkv5RSl5n70tQ,한국인들은 절대 안사는 화장품 КОСМЕТИКА КОТОРАЯ КОРЕЯНКИ ВО...,подписаться: https://goo.gl/3QbeUf -----------...,https://i.ytimg.com/vi/Wi49rKRM85U/default.jpg,120,...,https://i.ytimg.com/vi/Wi49rKRM85U/mqdefault.jpg,320,180,https://i.ytimg.com/vi/Wi49rKRM85U/hqdefault.jpg,480,360,Kyunghamin твоя корейская онни,none,2022-06-20T12:57:48Z,NaN
1,2,NjKFlYBBEw9zdA5NO0pNPxZQWW4,youtube#video,1ErvLvQoYxc,2019,UC6C0oFYlpKkFGG-ykqM4fYw,ТЫ ДОЛЖНА ЭТО ПОПРОБОВАТЬ! ЛУЧШАЯ КОРЕЙСКАЯ КО...,"Хорошие мои, сегодня я расскажу вам про свой к...",https://i.ytimg.com/vi/1ErvLvQoYxc/default.jpg,120,...,https://i.ytimg.com/vi/1ErvLvQoYxc/mqdefault.jpg,320,180,https://i.ytimg.com/vi/1ErvLvQoYxc/hqdefault.jpg,480,360,Pineapple Soul,none,2019-03-21T10:23:49Z,NaN
2,3,ZkrSJlyz8XsSTJv1_ejDaFtSKyM,youtube#video,aiY0ymL25Pw,2021,UCiT7bPorr4rKJU8WFBZA9Og,РЕАКЦИЯ дерматолога: 10 этапов корейского уход...,Мой телеграм: https://t.me/dr_amina_pirmanova ...,https://i.ytimg.com/vi/aiY0ymL25Pw/default.jpg,120,...,https://i.ytimg.com/vi/aiY0ymL25Pw/mqdefault.jpg,320,180,https://i.ytimg.com/vi/aiY0ymL25Pw/hqdefault.jpg,480,360,Amina Pirmanova,none,2021-08-20T08:51:18Z,NaN
3,4,uSLsO6jgTFGew3isXWNKN2IEIu0,youtube#video,JA9E4TCVsDU,2015,UCNazbf-kLY6E8TO_o48ZExQ,КАК Я ИСПОРТИЛА КОЖУ: КОРЕЙСКАЯ КОСМЕТИКА\Tony...,Привет! Корейская косметика снова набирает обо...,https://i.ytimg.com/vi/JA9E4TCVsDU/default.jpg,120,...,https://i.ytimg.com/vi/JA9E4TCVsDU/mqdefault.jpg,320,180,https://i.ytimg.com/vi/JA9E4TCVsDU/hqdefault.jpg,480,360,Julie Art,none,2015-12-06T14:30:40Z,NaN
4,5,6kkxWnMmA0vx1FI_Ur5fXgKuR6k,youtube#video,NAeXVpMTiT4,2018,UCmXGqfEsluA1fID55bVexjw,🇰🇷 НЕВЕРОЯТНАЯ КОРЕЙСКАЯ КОСМЕТИКА! Топовые шт...,Магазин корейской косметики https://goo.gl/UJG...,https://i.ytimg.com/vi/NAeXVpMTiT4/default.jpg,120,...,https://i.ytimg.com/vi/NAeXVpMTiT4/mqdefault.jpg,320,180,https://i.ytimg.com/vi/NAeXVpMTiT4/hqdefault.jpg,480,360,Tanya StreLove,none,2018-04-20T15:19:56Z,NaN


In [ ]:
# 18.1
# Первый заход БЕЗ аргумента order для видео
request = youtube.search().list(
        part ="snippet",
        maxResults=50,
        q = query,
        regionCode ="RU",
        type = type_query
)
response = request.execute()

df_supplemented = pd.json_normalize(response['items'])

    # Цикл для прохода по всем следующим страницам с выдачей:
i = 0
while 'nextPageToken' in response.keys():
    request = youtube.search().list(
            part ="snippet",
            maxResults=50,
            q = query,
            regionCode ="RU",
            type = type_query,
            pageToken = response['nextPageToken']
        )
    response = request.execute()

    print(f'Итерация БЕЗ включения аргумента order №{i}')
    i += 1

    df_additional = pd.json_normalize(response['items'])
    df_supplemented = pd.concat([df_supplemented, df_additional])



print(f"Искомых объектов {response['pageInfo']['totalResults']}", \
      f"а найденных БЕЗ включения каких-либо значений аргумента order {len(df_supplemented.drop_duplicates(f'id.{type_query}Id'))}")